In [77]:
import pandas as pd
import numpy as np
import random

In [78]:
waypoint_distances = {}
waypoint_durations = {}
all_waypoints = set()

waypoint_data = pd.read_csv("my-waypoints-dist-dur.tsv", sep="\t", encoding="ISO-8859-1")

for i, row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
    waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
    all_waypoints.update([row.waypoint1, row.waypoint2])

In [79]:
homebase = 'Gronnegata 11, Oslo'

In [80]:
# my_tuple = ('Nobels gate 32, Oslo', 'Bygdoy, Oslo, Oslo', 'Huk Aveny 35, Oslo', 
#             'Kongeveien 5, Oslo', 'nan, Oslo', 'Bygdoeynesveien 39, Oslo', 'Museumsveien 10, Oslo',
#             'Universitetsgaten 13 (city centre), Oslo', 'Kirsten Flagstads Pl. 1, Oslo')
# new_tupla = (homebase,) + my_tuple
# print(new_tupla)

In [93]:
a = ('ciao',)
b = a + ('b', 'c') + a + ('d', 'e')

for index in range(len(b)):
    print(index)
    print(b[index])
    print(a[0])
    print(b[index] == a[0])
    print()

0
ciao
ciao
True

1
b
ciao
False

2
c
ciao
False

3
ciao
ciao
True

4
d
ciao
False

5
e
ciao
False



In [95]:
def compute_fitness(homebase, solution, pop_index):
    """
        This function returns the total distance traveled on the current road trip.

        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """

    # solution_fitness = 0.0
    solution_fitness1 = 0.0
    solution_fitness2 = 0.0
    solution = (homebase,) + solution[:pop_index] + (homebase,) + solution[pop_index:]

    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        # solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
        # new decision function:
        if index < int(len(solution)/2):
            solution_fitness1 += waypoint_distances[frozenset([waypoint1, waypoint2])]
        else:
            solution_fitness2 += waypoint_distances[frozenset([waypoint1, waypoint2])]
            
    solution_fitness = np.sqrt(solution_fitness1 + solution_fitness2) + np.power(np.abs(solution_fitness1 - solution_fitness2), 2/3)

    return solution_fitness

In [82]:
def generate_random_agent(homebase):
    """
        Creates a random road trip from the waypoints.
    """

    new_random_agent = list(all_waypoints)
    new_random_agent.remove(homebase)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

In [83]:
def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.

        A point mutation swaps the order of two waypoints in the road trip.
    """

    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)

    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]

    return tuple(agent_genome)

In [84]:
def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.

        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """

    agent_genome = list(agent_genome)

    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)

    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]

    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]

    return tuple(agent_genome)

In [85]:
def generate_random_population(homebase, pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """

    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent(homebase))
    return random_population

In [96]:
def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.

        `generations` and `population_size` must be a multiple of 10.
    """

    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    pop_half_ss = int(population_subset_size/2)
    print(population_subset_size)

    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(homebase, population_size)
    # print(population)

    # For `generations` number of repetitions...
    for generation in range(generations):

        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(homebase, agent_genome, pop_half_ss)

        # Take the top 10% shortest road trips and produce offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):

            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                # complete_agent_genome = (homebase,) + agent_genome
                complete_agent_genome = (homebase,) + agent_genome[:pop_half_ss] + (homebase,) + agent_genome[pop_half_ss:]
                print(complete_agent_genome)
                print("")

            # Create 1 exact copy of each of the top road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))

            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population

In [97]:
run_genetic_algorithm(generations=5000, population_size=100)

10
Generation 0 best: 243 | Unique genomes: 100
('Gronnegata 11, Oslo', 'Toeyengata 53, Oslo', 'Astrup Fearnley Museet, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Sognsveien 235, Oslo', 'Bygdoeynesveien 39, Oslo', 'Gronnegata 11, Oslo', 'Fridtjof Nansens plass, Oslo', 'Kjelsaasveien 143, Oslo', 'Nobels gate 32, Oslo', 'Det Kongelige Slott, Oslo', 'Myntgata 5, Oslo')

Generation 500 best: 205 | Unique genomes: 76
('Gronnegata 11, Oslo', 'Bygdoeynesveien 39, Oslo', 'Nobels gate 32, Oslo', 'Toeyengata 53, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Det Kongelige Slott, Oslo', 'Gronnegata 11, Oslo', 'Kjelsaasveien 143, Oslo', 'Sognsveien 235, Oslo', 'Astrup Fearnley Museet, Oslo', 'Myntgata 5, Oslo', 'Fridtjof Nansens plass, Oslo')

Generation 1000 best: 198 | Unique genomes: 70
('Gronnegata 11, Oslo', 'Sognsveien 235, Oslo', 'Kjelsaasveien 143, Oslo', 'Toeyengata 53, Oslo', 'Astrup Fearnley Museet, Oslo', 'Myntgata 5, Oslo', 'Gronnegata 11, Oslo', 'Kirsten Flagstads Pl. 1, Oslo', 'Nobels gate 